In [3]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm
import dask.dataframe as dd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)
from dask.distributed import Client
client = Client()


/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
df_train = dd.read_csv('../input/train.csv')
df_test = dd.read_csv('../input/test.csv')
df_hist_trans = dd.read_csv('../input/historical_transactions.csv')
df_new_merchant_trans = dd.read_csv('../input/new_merchant_transactions.csv')

In [3]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'] = df['category_2'].fillna(1.0)
    df['category_3'] = df['category_3'].fillna('A')
    df['merchant_id'] = df['merchant_id'].fillna('M_ID_00a6ca8a8a')


In [4]:
df_hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [4]:
holidays = np.asarray([pd.to_datetime(date) for date in ["Jan 01, 2017","Feb 27, 2017","Feb 28, 2017","Mar 01, 2017","Apr 14, 2017","Apr 16, 2017","Apr 21, 2017","May 01, 2017","Jun 15, 2017","Sep 07, 2017","Oct 12, 2017","Nov 02, 2017","Nov 15, 2017","Dec 25, 2017","1 Jan 2018","12 Feb 2018","13 Feb 2018","30 Mar 2018","21 Apr 2018","1 May 2018","31 May 2018","7 Sep 2018","12 Oct 2018","2 Nov 2018","15 Nov 2018","20 Nov 2018","25 Dec 2018"]])
seasons = [0,0,0,1,1,1,2,2,2,4,4,4]
tqdm.pandas()
def closest_holiday_days(x):
    deltas = holidays - x
    vect_abs = np.vectorize(abs)
    vect_days = np.vectorize(lambda x: x.days)
    absed = vect_abs(vect_days(deltas))
    return min(absed)

In [8]:
a = pd.to_datetime("Jan 01, 2017")
%timeit closest_holiday_days(a)

457 µs ± 4.45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:

holidays = np.asarray([pd.to_datetime(date) for date in ["Jan 01, 2017","Feb 27, 2017","Feb 28, 2017","Mar 01, 2017","Apr 14, 2017","Apr 16, 2017","Apr 21, 2017","May 01, 2017","Jun 15, 2017","Sep 07, 2017","Oct 12, 2017","Nov 02, 2017","Nov 15, 2017","Dec 25, 2017","1 Jan 2018","12 Feb 2018","13 Feb 2018","30 Mar 2018","21 Apr 2018","1 May 2018","31 May 2018","7 Sep 2018","12 Oct 2018","2 Nov 2018","15 Nov 2018","20 Nov 2018","25 Dec 2018"]])
seasons = [0,0,0,1,1,1,2,2,2,4,4,4]
tqdm.pandas()
def closest_holiday_days(x):
    deltas = holidays - x
    vect_abs = np.vectorize(abs)
    vect_days = np.vectorize(lambda x: x.days)
    absed = vect_abs(vect_days(deltas))
    return min(absed)

for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = dd.to_datetime(df['purchase_date'])    
    print("processsed purchase_date")
    df['year'] = df['purchase_date'].dt.year
    print("year") 
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    print("wof")     
    df['month'] = df['purchase_date'].dt.month
    print("month")         
    df['season'] = df['month'].apply(lambda x:seasons[x-1])
    print("season")    
    df['closest_date_to_holiday'] = df['purchase_date'].apply(closest_holiday_days)
    print("holiday")             
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    print("dayofweek")    
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    print("weekend")        
    df['hour'] = df['purchase_date'].dt.hour
    print("hour")    
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    print("AF")            
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    print("cat_1")    
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    print("month_diff")                
    df['month_diff'] += df['month_lag']

NameError: name 'df_hist_trans' is not defined

In [6]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [7]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['season'] = ['sum','max','min','mean','var']
aggs['closest_date_to_holiday'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

df_hist_trans = df_hist_trans.compute()

for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans['purchase_amount'].map(df_hist_trans.groupby([col])['purchase_amount'].mean().compute())
    aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:64532, threads: 1>>
Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 355, in catch_zombie
    yield
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 394, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot(self.pid)
ProcessLookupError: [Errno 3] No such process

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/ioloop.py", line 1229, in _run
    return self.callback()
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/nanny.py", line 264, in memory_monitor
    memory = proc.memory_info().rss
  File "/

distributed.nanny - WARNING - Worker process 35977 was killed by unknown signal
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:64527, threads: 1>>
Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 355, in catch_zombie
    yield
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 394, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot(self.pid)
ProcessLookupError: [Errno 3] No such process

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/ioloop.py", line 1229, in _run
    return self.callback()
  File "/Users/raghavan/anaconda2/envs/p

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:64525, threads: 1>>
Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 355, in catch_zombie
    yield
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 394, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot(self.pid)
ProcessLookupError: [Errno 3] No such process

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/ioloop.py", line 1229, in _run
    return self.callback()
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/nanny.py", line 264, in memory_monitor
    memory = proc.memory_info().rss
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_commo

distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:64527'], ('assign-2575c52cc747cd0a18d909996558709b', 20)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:64527'], ('assign-2575c52cc747cd0a18d909996558709b', 39)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:64525'], ('assign-2575c52cc747cd0a18d909996558709b', 11)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:64527'], ('assign-2575c52cc747cd0a18d909996558709b', 37)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:64525'], ('assign-2575c52cc747cd0a18d909996558709b', 19)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://127.0.0.1:64527'], ('assign-2575c52cc747cd0a18d909996558709b', 28)
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:64524, threads: 1>>
Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 355, in catch_zombie
    yield
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 394, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot(self.pid)
ProcessLookupError: [Errno 3] No such process

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/ioloop.py", line 1229, in _run
    return self.callback()
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/nanny.py", line 264, in memory_monitor
    memory = proc.memory_info().rss
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_commo

...

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:64535, threads: 1>>
Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 355, in catch_zombie
    yield
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 394, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot(self.pid)
ProcessLookupError: [Errno 3] No such process

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/ioloop.py", line 1229, in _run
    return self.callback()
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/nanny.py", line 264, in memory_monitor
    memory = proc.memory_info().rss
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_commo

tornado.application - ERROR - Exception in callback <bound method Nanny.memory_monitor of <Nanny: tcp://127.0.0.1:64521, threads: 1>>
Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 355, in catch_zombie
    yield
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_psosx.py", line 394, in _get_pidtaskinfo
    ret = cext.proc_pidtaskinfo_oneshot(self.pid)
ProcessLookupError: [Errno 3] No such process

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/tornado/ioloop.py", line 1229, in _run
    return self.callback()
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/nanny.py", line 264, in memory_monitor
    memory = proc.memory_info().rss
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/psutil/_commo

Traceback (most recent call last):
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-c6804191a537>", line 17, in <module>
    df_hist_trans = df_hist_trans.compute()
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/dask/base.py", line 156, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/dask/base.py", line 397, in compute
    results = schedule(dsk, keys, **kwargs)
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/client.py", line 2318, in get
    direct=direct)
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distributed/client.py", line 1652, in gather
    asynchronous=asynchronous)
  File "/Users/raghavan/anaconda2/envs/py36/lib/python3.6/site-packages/distri

KeyboardInterrupt: 

distributed.nanny - WARNING - Worker process 36637 was killed by unknown signal
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['season'] = ['sum','max','min','mean','var']
aggs['closest_date_to_holiday'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

for col in ['category_2','category_3']:
    df_new_merchant_trans.groupby([col])['purchase_amount'].mean()
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans["purchase_amount"].map(df_new_merchant_trans.groupby([col])['purchase_amount'].mean().compute())
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new_hist',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

In [ ]:
del df_hist_trans;gc.collect()
del df_new_merchant_trans;gc.collect()
df_train.head(5)


In [ ]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

In [ ]:
for df in [df_train,df_test]:
    df['first_active_month'] = df['first_active_month'].map(lambda x: pd.to_datetime(x, errors='coerce'))
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)


In [ ]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = df_train['target']
del df_train['target']

In [ ]:
df_train = df_train.compute()
df_test = df_test.compute()

In [20]:
x = df_train.columns.to_series().groupby(df.dtypes).groups
x

{dtype('<M8[ns]'): Index(['first_active_month'], dtype='object'),
 dtype('int64'): Index(['hist_month_nunique', 'hist_hour_nunique', 'hist_weekofyear_nunique',
        'hist_dayofweek_nunique', 'hist_year_nunique',
        'hist_subsector_id_nunique', 'hist_merchant_id_nunique',
        'hist_merchant_category_id_nunique', 'hist_installments_sum',
        'hist_installments_max', 'hist_installments_min', 'hist_season_sum',
        'hist_season_max', 'hist_season_min',
        'hist_closest_date_to_holiday_sum', 'hist_closest_date_to_holiday_max',
        'hist_closest_date_to_holiday_min', 'hist_month_lag_max',
        'hist_month_lag_min', 'hist_authorized_flag_sum', 'hist_weekend_sum',
        'hist_category_1_sum', 'hist_card_id_size', 'hist_purchase_date_diff',
        'hist_purchase_date_uptonow'],
       dtype='object'),
 dtype('float64'): Index(['feature_1', 'feature_2', 'feature_3', 'hist_purchase_amount_sum',
        'hist_purchase_amount_max', 'hist_purchase_amount_min',
    

In [21]:
from sklearn.preprocessing import PolynomialFeatures
# df_train["card_id"] = df_train_string["card_id"]
df_train_string = df_train[["card_id","first_active_month"]]

del df_train["card_id"]
del df_train["first_active_month"]

# df_train = PolynomialFeatures(degree=2).fit(df_train)
# df_test = PolynomialFeatures(degree=2).fit(df_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [22]:
df_train[df_train.isna()]

,feature_1,feature_2,feature_3,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,hist_subsector_id_nunique,hist_merchant_id_nunique,...,new_hist_purchase_date_uptonow,outliers,dayofweek,weekofyear,month,elapsed_time,hist_first_buy,new_hist_first_buy,card_id_total,purchase_amount_total
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

np.sqrt(mean_squared_error(oof, target))

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')


In [ ]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission_poly.csv", index=False)

In [ ]:
# holidays = [1,3,2,3,5]
print(holidays)
a = [date for date in holidays]

In [ ]:
min(df_new_merchant_trans["purchase_date"])

In [ ]:
max(df_new_merchant_trans["purchase_date"])

In [ ]:
df_new_merchant_trans.dtypes

In [ ]:
[pd.to_datetime(date) for date in ["Jan 01, 2017","Feb 27, 2017","Feb 28, 2017","Mar 01, 2017","Apr 14, 2017","Apr 16, 2017","Apr 21, 2017","May 01, 2017","Jun 15, 2017","Sep 07, 2017","Oct 12, 2017","Nov 02, 2017","Nov 15, 2017","Dec 25, 2017","1 Jan 2018","12 Feb 2018","13 Feb 2018","30 Mar 2018","21 Apr 2018","1 May 2018","31 May 2018","7 Sep 2018","12 Oct 2018","2 Nov 2018","15 Nov 2018","20 Nov 2018","25 Dec 2018"]]

In [ ]:
(pd.to_datetime("Apr 21, 2017") - pd.to_datetime('2017-01-01 00:00:08')).days